## 1. 問題分析結果

### 競技プログラミング視点

* 連続一様分布なので **確率＝面積比**（長方形 `A×B` 内で `x+y<C` を満たす領域の面積 / `A*B`）。
* 面積は `C` と `A,B` の大小関係で **場合分けすれば O(1)** で計算可能。
* 浮動小数は不要（境界 `x+y=C` は測度 0 なので `<` と `<=` は同じ）。**整数だけで 2 倍面積**を持つと安全。

### 業務開発視点

* 場合分けを関数化し、`gcd` で既約分数にするだけなので保守性が高い。
* 入力制約（正整数、上限 10000）を前提にしつつ、異常値（`C<=0`）にも耐える実装にする。

### Python特有考慮（CPython）

* `math.gcd` は高速。分岐も少なく、テストケース数が多くてもボトルネックになりにくい。
* **float を使わない**（丸め誤差で WA 回避）。

---

## 2. アルゴリズム比較表

| アプローチ                  | 時間計算量 | 空間計算量 | Python実装コスト | 可読性 | 標準ライブラリ活用  | CPython最適化 | 備考                      |
| ---------------------- | ----: | ----: | ----------: | --: | ---------- | ---------- | ----------------------- |
| 方法A: 幾何（場合分けで面積を閉形式計算） |  O(1) |  O(1) |           低 | ★★★ | `math.gcd` | 適          | **採用**。整数で `2*area` を持つ |
| 方法B: 数値積分（刻みで近似）       |  O(K) |  O(1) |           中 | ★★☆ | なし         | 不適         | 近似誤差のリスク                |
| 方法C: モンテカルロ            |  O(K) |  O(1) |           低 | ★★☆ | `random`   | 不適         | 乱数なので確率がブレる（不可）         |

---

## 3. 採用アルゴリズムと根拠

* **選択理由**: 面積が三角形/台形の組み合わせになり、`C` の位置で決まるため **閉形式**で求まる。
* **Python最適化戦略**:

  * 面積 `area` そのままだと `1/2` が出るので、`D = 2*area`（整数）として保持。
  * 最終確率は `D / (2*A*B)` を `gcd` で約分。
* **トレードオフ**: 数学的場合分けは必要だが、実装は短く高速・正確。

---

## 4. 検証（境界観点）

* `C <= 0` → 0/1
* `C >= A+B` → 1/1
* `C <= min(A,B)` → 三角形 `C^2/2`
* `min(A,B) < C <= max(A,B)` → 台形 `a*C - a^2/2`
* `max(A,B) < C < A+B` → 反対側の欠けた三角形を引いた形（閉形式）

---

## HackerRankでの回答フォーマット（solve 関数のみ実装）

```python
import math
import os
import random
import re
import sys


def _reduce_fraction(numer: int, denom: int) -> str:
    if numer == 0:
        return "0/1"
    g = math.gcd(numer, denom)
    return f"{numer // g}/{denom // g}"


def _probability_doubled_area(a: int, b: int, c: int) -> int:
    """
    Returns D = 2 * area where area is the measure of {(x,y) in [0,a]x[0,b] : x+y < c}.
    Uses integer arithmetic only.
    """
    if c <= 0:
        return 0

    # Sort to simplify cases: a <= b
    if a > b:
        a, b = b, a

    if c >= a + b:
        return 2 * a * b

    if c <= a:
        # area = c^2 / 2  => D = c^2
        return c * c

    if c <= b:
        # area = ∫_0^a (c - x) dx = a*c - a^2/2  => D = 2ac - a^2
        return 2 * a * c - a * a

    # b < c < a+b
    # D = 2c(a+b) - c^2 - a^2 - b^2
    return 2 * c * (a + b) - c * c - a * a - b * b


def solve_production(a: int, b: int, c: int) -> str:
    """
    業務開発向け（型安全・読みやすさ重視）
    Time Complexity: O(1)
    Space Complexity: O(1)
    """
    if not (isinstance(a, int) and isinstance(b, int) and isinstance(c, int)):
        raise TypeError("a, b, c must be integers")
    if a <= 0 or b <= 0:
        raise ValueError("a and b must be positive")

    doubled_area = _probability_doubled_area(a, b, c)
    denom = 2 * a * b
    return _reduce_fraction(doubled_area, denom)


def solve_competitive(a: int, b: int, c: int) -> str:
    """
    競技プログラミング向け（最小限・高速）
    Time Complexity: O(1)
    Space Complexity: O(1)
    """
    doubled_area = _probability_doubled_area(a, b, c)
    denom = 2 * a * b
    if doubled_area == 0:
        return "0/1"
    g = math.gcd(doubled_area, denom)
    return f"{doubled_area // g}/{denom // g}"


def solve(a: int, b: int, c: int) -> str:
    # HackerRank から呼ばれる本体（競技用を採用）
    return solve_competitive(a, b, c)
```


# Random number generator - 一様乱数 2 変数の和がしきい値未満となる確率

## 目次（TOC）

* [概要](#overview)
* [アルゴリズム要点](#tldr)
* [図解](#figures)
* [証明のスケッチ](#proof)
* [計算量](#complexity)
* [Python 実装](#impl)
* [CPython 最適化ポイント](#cpython)
* [エッジケースと検証](#edgecases)
* [FAQ](#faq)

---

<h2 id="overview">概要</h2>

**{{PLATFORM}}: {{PROBLEM_ID}} Random number generator**

理想的な乱数生成器により、独立な連続一様乱数

* $x \sim \mathrm{Uniform}(0, A)$
* $y \sim \mathrm{Uniform}(0, B)$

を生成する。正の整数 $C$ に対し、条件 $x + y < C$ を満たす確率を **既約分数**で求める。

### 入出力（簡潔）

* 入力: テストケース数 $N$、各ケースで整数 $A, B, C$
* 出力: 各ケースの確率を `p/q`（$\gcd(p,q)=1$）で出力

### 代表例

* $A=1,B=1,C=1 \Rightarrow 1/2$
* $A=1,B=1,C=2 \Rightarrow 1/1$
* $A=1,B=1,C=3 \Rightarrow 1/1$

---

<h2 id="tldr">アルゴリズム要点</h2>

### 観察

確率密度は長方形領域 $[0,A] \times [0,B]$ 上で一様なので、

$$
\Pr(x+y<C) ;=; \frac{\text{面積}\bigl({(x,y)\in[0,A]\times[0,B] \mid x+y<C}\bigr)}{A\cdot B}
$$

境界 $x+y=C$ は面積 $0$ のため、$x+y<C$ と $x+y\le C$ の違いは確率に影響しない。

### 方針

* 面積を場合分けで閉形式計算する（$O(1)$）
* 分数を正確に扱うため、面積そのものではなく **$D = 2\cdot \text{面積}$** を整数で計算
* 最終確率は $\dfrac{D}{2AB}$ を `gcd` で約分

### 面積の閉形式（$a=\min(A,B),; b=\max(A,B)$ とする）

* $C \le 0$: $D=0$
* $C \ge a+b$: $D=2ab$
* $0 < C \le a$: 三角形で $D=C^2$
* $a < C \le b$: 台形で $D=2aC-a^2$
* $b < C < a+b$: 反対側の欠けを引いて
  $$
  D = 2C(a+b) - C^2 - a^2 - b^2
  $$

---

<h2 id="figures">図解</h2>

```mermaid
flowchart TD
  Start[開始] --> Input[入力 A B C]
  Input --> Normalize[正規化 a 最小 b 最大]
  Normalize --> CaseCheck[場合分け]
  CaseCheck --> AreaCalc[2倍面積 D を整数で計算]
  AreaCalc --> Reduce[gcd で既約化]
  Reduce --> Output[分数 p slash q を返す]
```

上図は「面積比＝確率」の計算フローを示す。実装は分岐ごとに $D$ を算出し、最後に $\dfrac{D}{2AB}$ を約分して文字列化する。

---

<h2 id="proof">証明のスケッチ</h2>

### 幾何モデル

* 全体領域は長方形 $R=[0,A]\times[0,B]$、面積は $AB$
* 求めたい領域は半平面 $H={(x,y)\mid x+y<C}$ と長方形の交わり $R\cap H$

一様分布のため、

$$
\Pr(x+y<C) = \frac{\mathrm{Area}(R\cap H)}{\mathrm{Area}(R)} = \frac{\mathrm{Area}(R\cap H)}{AB}
$$

### 場合分けの正当性

$a=\min(A,B)$、$b=\max(A,B)$ とし、直線 $x+y=C$ の切り取り方が変化する臨界点は $C=a$ と $C=b$ と $C=a+b$。

1. $0 < C \le a$
   直線は両軸上で $(C,0),(0,C)$ を結び、長方形内の領域は直角三角形。
   面積は $\dfrac{C^2}{2}$ なので $D=2\cdot\dfrac{C^2}{2}=C^2$。

2. $a < C \le b$
   $x\in[0,a]$ では上限 $y=C-x$ が $[0,b]$ 内に収まり、全区間で切り取りが成立。
   面積は
   $$
   \int_0^a (C-x),dx = aC - \frac{a^2}{2}
   $$
   よって $D=2aC-a^2$。

3. $b < C < a+b$
   直線は上辺・右辺にも交差し、全体 $AB$ から右上の直角三角形を引ける。
   引く三角形の辺長は $(a+b-C)$ と同型になる形へ整理でき、最終的に
   $$
   D = 2C(a+b) - C^2 - a^2 - b^2
   $$
   が得られる（既知の台形分解と対称性で導出可能）。

4. $C \ge a+b$
   長方形全体が条件を満たし、確率は $1$。

### 終了性

分岐は定数個で、各分岐は定数時間の算術演算のみ。よって必ず停止する。

---

<h2 id="complexity">計算量</h2>

* 時間計算量: $O(1)$（各テストケースあたり）
* 空間計算量: $O(1)$

---

<h2 id="impl">Python 実装</h2>

* HackerRank 形式: `solve(a, b, c) -> str` を完成
* **Pure**: 標準入出力は触らず、引数から結果文字列を返す
* 型注釈あり（Pylance 想定）

```python
from __future__ import annotations

import math
from typing import Final


def _reduce_fraction(numer: int, denom: int) -> str:
    """既約分数 `numer/denom` を文字列化して返す。"""
    if numer == 0:
        return "0/1"
    g: int = math.gcd(numer, denom)
    return f"{numer // g}/{denom // g}"


def _doubled_area(a: int, b: int, c: int) -> int:
    """
    D = 2 * area を整数で返す。
    area = Area({(x,y) in [0,a]x[0,b] | x+y < c}).

    分数を避けるため 2 倍面積で保持する:
      P = area / (a*b) = D / (2*a*b)
    """
    if c <= 0:
        return 0

    # 正規化: a <= b
    if a > b:
        a, b = b, a

    # C >= a+b なら全領域
    if c >= a + b:
        return 2 * a * b

    # 0 < C <= a: 三角形 area = C^2/2 => D = C^2
    if c <= a:
        return c * c

    # a < C <= b: 台形 area = a*C - a^2/2 => D = 2aC - a^2
    if c <= b:
        return 2 * a * c - a * a

    # b < C < a+b:
    # D = 2C(a+b) - C^2 - a^2 - b^2
    return 2 * c * (a + b) - c * c - a * a - b * b


def solve(a: int, b: int, c: int) -> str:
    """
    HackerRank が呼び出す関数。

    Args:
        a: 上限 A（正整数）
        b: 上限 B（正整数）
        c: しきい値 C（整数、問題文では正整数）

    Returns:
        既約分数 "p/q" 形式
    """
    # 問題制約上は正整数だが、堅牢性のため最低限の防御を入れる
    if a <= 0 or b <= 0:
        raise ValueError("a and b must be positive integers")

    d: int = _doubled_area(a, b, c)
    denom: int = 2 * a * b
    return _reduce_fraction(d, denom)
```

---

<h2 id="cpython">CPython 最適化ポイント</h2>

* 浮動小数を使わず整数のみで処理する（丸め誤差による WA 回避）
* `math.gcd` を使って既約化（高速で信頼性が高い）
* 場合分けは定数個で分岐も浅い（テストケースが多くても安定）

---

<h2 id="edgecases">エッジケースと検証</h2>

| ケース    | 条件                            | 期待する確率                             |
| ------ | ----------------------------- | ---------------------------------- |
| 下限     | $C \le 0$                     | $0/1$                              |
| 全領域    | $C \ge A+B$                   | $1/1$                              |
| 小さい三角形 | $0 < C \le \min(A,B)$         | $\dfrac{C^2}{2AB}$                 |
| 台形領域   | $\min(A,B) < C \le \max(A,B)$ | $\dfrac{2aC-a^2}{2AB}$             |
| 欠け三角形  | $\max(A,B) < C < A+B$         | $\dfrac{2C(a+b)-C^2-a^2-b^2}{2AB}$ |

追加の確認観点:

* $A=B$ の対称ケース（式が自然に一致する）
* $C=1$ や $C=A$ など臨界点（分岐境界でも連続に一致する）
* 最大制約 $A,B,C \le 10000$（$2AB$ や各式は $32$ bit を超えうるが Python の `int` なら安全）

---

<h2 id="faq">FAQ</h2>

**Q1. なぜ $2\cdot\text{面積}$ を計算するのですか？**
A. 三角形や台形で $\dfrac{1}{2}$ が頻出するため、$D=2\cdot\text{面積}$ を整数で持つと誤差なく簡単に実装できます。最終確率は $\dfrac{D}{2AB}$ です。

**Q2. 条件が $x+y\le C$ だったら結果は変わりますか？**
A. 変わりません。境界 $x+y=C$ は面積が $0$ なので確率に影響しません。
